## Snow Pit Layer Object and Initial Investigation

Initial research question: What is the relationship between snow pit layer hand hardness and grain form?

### Import libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xml.etree.ElementTree as ET

In [1]:
print('Hello World!') 

Hello World!


#Create layer object

In [3]:
## Don't need?
class Layer:
    # Translated from https://github.com/SnowpitData/AvscienceServer/blob/master/src/avscience/ppc/Layer.java
    def __init__(self, startDepth, endDepth, hardness1, hsuffix1, hardness2, hsuffix2, grainType, grainType1, grainSizeUnits1, grainSizeUnits2, grainSize, grainSuffix, grainSize1, grainSuffix1, waterContent, density1, density2, comments, layerNumber, fromTop):
        self.startDepth=startDepth   
        self.endDepth=endDepth
        self.hardness1=hardness1
        self.hsuffix1=hsuffix1
        self.hardness2=hardness2
        self.hsuffix2=hsuffix2
        self.grainType=grainType
        self.grainType1=grainType1
        self.grainSizeUnits1=grainSizeUnits1
        self.grainSizeUnits2=grainSizeUnits2
        self.grainSize=grainSize
        self.grainSuffix=grainSuffix
        self.grainSize1=grainSize1
        self.grainSuffix1=grainSuffix1
        self.waterContent=waterContent
        self.density1=density1
        self.density2=density2
        self.comments=comments
        self.layerNumber=layerNumber
        self.fromTop=fromTop
                 



In [4]:
# More promising approach

## Create layer object from caaml file format
class caaml_layer:
    def __init__(self, depthTop_uom, depthTop, thickness_uom, thickness, grainFormPrimary, grainFormSecondary, hardness, wetness):
        self.depthTop_uom=depthTop_uom
        self.depthTop=depthTop
        self.thickness_uom=thickness_uom
        self.thickness=thickness
        self.grainFormPrimary=grainFormPrimary
        self.grainFormSecondary=grainFormSecondary
        self.hardness=hardness
        self.wetness=wetness

## Convert to depth and thickness to cm
def convert_to_cm(depthTop_uom, depthTop, thickness_uom, thickness):
    if depthTop_uom == 'cm':
        depthTop = depthTop
    elif depthTop_uom == 'in':
        depthTop = depthTop * 2.54

    if thickness_uom == 'cm':
        thickness = thickness
    elif thickness_uom == 'in':
        thickness = thickness * 2.54

    return depthTop, thickness


## Create dictionary for primary grain form
## Reference: src/avscience/ppc/GrainTypeConvertor.java
grainFormPrimaryDict = {
    'PP': 'Precipitation particles',
    'DF': 'Decomposing and fragmented precipitation particles',
    'RG': 'Rounded grains',
    'FC': 'Faceted crystals',
    'DH': 'Depth hoar',
    'SH': 'Surface hoar',
    'MF': 'Melt forms',
    'IF': 'Ice formations',
    'MM': 'Machine made Snow'
}

## Create dictionary for secondary grain form
## Reference: src/avscience/ppc/GrainTypeConvertor.java
grainFormSecondaryDict = {
    'PPgp': 'Graupel',
    'PPco': 'Columns',
    'PPpl': 'Plates',
    'PPnd': 'Needles',
    'PPsd': 'Stellars, Dendrites',
    'PPir': 'Irregular crystals',
    'PPip': 'Ice pellets',
    'PPrm': 'Rime',
    'DFdc': 'Partly decomposed precipitation particles',
    'DFbk': 'Wind-broken precipitation particles',
    'RGsr': 'Small rounded particles',
    'RGlr': 'Large rounded particles',
    'RGwp': 'Wind packed',
    'RGxf': 'Faceted rounded particles',
    'FCso': 'Solid faceted particles',
    'FCsf': 'Near surface faceted particles',
    'FCxr': 'Rounding faceted particles',
    'DHcp': 'Hollow cups',
    'DHpr': 'Hollow prisms',
    'DHch': 'Chains of depth hoar',
    'DHla': 'Large striated crystals',
    'DHxr': 'Rounding depth hoar',
    'SHsu': 'Surface hoar crystals',
    'SHcv': 'Cavity or crevasse hoar',
    'SHxr': 'Rounding surface hoar',
    'MFcl': 'Clustered rounded grains',
    'MFpc': 'Rounded polycrystals',
    'MFsl': 'Slush',
    'MFcr': 'Melt-freeze crust',
    'IFil': 'Ice layer',
    'IFic': 'Ice column',
    'IFbi': 'Basal ice',
    'IFrc': 'Rain crust',
    'IFsc': 'Sun crust',
    'MMrp': 'Round polycrystalline particles',
    'MMci': 'Crushed ice particles',
}

## Create dictionary for hardness
## Reference: src/avscience/ppc/Layer.java
hardnessDict = {
    'F': 'Fist',
    '4F': '4 Fingers',
    '1F': '1 Finger',
    'P': 'Pencil',
    'K': 'Knife',
    'I': 'Ice'
}

## Create dictionary for wetness
## Reference: src/avscience/ppc/GrainTypeConvertor.java
wetnessDict = {
    'D': 'Dry',
    'M': 'Moist',
    'W': 'Wet',
    'U': 'Very wet'
}

In [12]:
## Read in data from caaml files

# Define the path to snowpits folder
folder_path = "snowpits_200_MT" # The latest 200 snow pits from MT

# Create a list of all CAAML files in the folder
caaml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

#print(caaml_files)
file1=caaml_files[0]
print(file1)


snowpits-66387-caaml.xml


In [19]:
# Parse file1
#https://www.datacamp.com/tutorial/python-xml-elementtree

file_path = "snowpits_200_MT\snowpits-66387-caaml.xml"
tree = ET.parse(file_path)
root = tree.getroot()

# Define the namespace (CAAML files typically use one)
ns = {'caaml': 'http://caaml.org/Schemas/V5.0/Profiles/SnowProfileIACS'}

# Get basic snowpit info
profile = root.find('.//caaml:snowProfileResultsOf', ns)
if profile:
    # Get date and location
    date = profile.find('.//caaml:timePosition', ns).text
    location = profile.find('.//caaml:pointLocation', ns)
    lat = location.find('.//caaml:latitude', ns).text
    lon = location.find('.//caaml:longitude', ns).text
    
    print(f"Date: {date}")
    print(f"Location: {lat}, {lon}")

# Get layers
layers = root.findall('.//caaml:stratProfile/caaml:Layer', ns)
for layer in layers:
    # Get depth and thickness
    depth = layer.find('.//caaml:depthTop', ns).text
    thickness = layer.find('.//caaml:thickness', ns).text
    
    # Get grain type and size
    grain_form = layer.find('.//caaml:grainFormPrimary', ns).text
    grain_size = layer.find('.//caaml:grainSize/caaml:Components/caaml:avg', ns).text
    
    print(f"\nLayer at depth {depth}cm:")
    print(f"Thickness: {thickness}cm")
    print(f"Grain form: {grain_form}")
    print(f"Average grain size: {grain_size}mm")

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_41196\23586216.py:2: SyntaxWarning: invalid escape sequence '\s'
  file_path = "snowpits_200_MT\snowpits-66387-caaml.xml"


Read in data from 10 snow pits and create table showing how many layers there are of each crystal form. 
1. Histogram of primary and secondary grain forms
2. Histogram of hardness
3. Boxplot of primary gram form vs hardness
4. Hardness vs wetness